<H1> Beam Load </H1> <br> 
Determine the beam height to support a 12' span with a specific load  <br> 
<hr> 
<H2>Modification history </H2>
<table>
    <tr>
        <th> Modified </th>
        <th> BY </th>
        <th> Reason </th>
    </tr>
    <tr>
        <td> 01-July-24</td>
        <td> CBL </td>
        <td> Original </td>
    </tr>
    <tr>
        <td> 02-Jul-24</td>
        <td> CBL</td>
        <td> Validated formulas, values looking correct.</td>
    </tr>
</table>

<hr> 
<H2> References </H2> 
<a href="https://en.m.wikipedia.org/wiki/Section_modulus"> Section Modulus for various geometries </a> <br>
<a href="https://skyciv.com/docs/tutorials/section-tutorials/moment-of-inertia-of-a-rectangle/#:~:text=In%20summary%2C%20the%20formula%20for,%E2%81%84%2012%20–%20bd³%20%E2%81%84%2012."> Area moment of inerta for rectangle </a><br> 
<a href="https://www.engineeringtoolbox.com/area-moment-inertia-d_1328.html"> Area moment of inerta for rectangular beam </a><br>
<a href="https://www.weyerhaeuser.com/blog/how-to-easily-calculate-allowable-deflection/"> Allowable deflections </a><br>
<a href="https://www.engineeringtoolbox.com/timber-mechanical-properties-d_1789.html"> Youngs modulus for timber </a><br>
<a href="https://skyciv.com/steel-i-beam-sizes/"> Steel I-Beam cross section values </a><br>
<a href="https://www.omnicalculator.com/construction/beam-deflection"> Deflections </a><br>
<a href="https://amesweb.info/section/i-beam-moment-of-inertia-calculator.aspx"> Area moment of intertia for i-beam </a>
<hr> 
This is an exterior wall, the deflection should be no more than: ${L\over{120}}$ to ${H\over{240}}$

In [1]:
# includes
import numpy as np
import matplotlib.pyplot as plt
import math

Area moment of inertia for rectangle: <br> 
<center> $I_{xx}={BD^3\over{12}} , I_{yy}={B^3D\over{12}}$ </center>
<br>
where: B is width and D is height. <br> 
<img src="https://www.engineeringtoolbox.com/docs/documents/1328/geometric_sections_rectangle-Model.png"> 

In [2]:
def AreaMomentBeam(W, H):
    """
    This is for a retangular homogenious beam
    @param W - width
    @param H - height
    Returns Area moment of inertia section properties. in units^4 
    TESTED: 01-Jul-24
    """
    rv_xx = W * np.power(H,3)/12
    rv_yy = np.power(W,3) * H/12
    return rv_xx,rv_yy

For an I-Beam the Area Moment of inertia is: <br>
<center> $I_x = ({ah^3\over{12}} + {b\over{12}}(H^3-h^3)$</center><br>
<center> $I_y = ({a^3h\over{12}} + {b^3\over{12}}(H-h)$</center>
<hr> 
<img src="https://www.engineeringtoolbox.com/docs/documents/1328/geometric_sections_symmetrical_shape-Model.png">

In [3]:
def AreaMomentIBeam(H1, H2, a, b ):
    """
    This is for an I-beam
    @param H1 - outer face to face height
    @param H2 - web height
    @param a  - web thickness
    @param b  - base width
    Returns Area moment of inertia section properties. in units^4 
    this reference is different. https://amesweb.info/section/i-beam-moment-of-inertia-calculator.aspx
    TESTED: 
    """
    rv_xx = a * np.power(H2,3)/12 + (b/12) * (np.power(H1,3) - np.power(H2,3))
    rv_yy = np.power(a,3) * H2/12 + (np.power(b,3)/12)*(H1-H2)
    return rv_xx,rv_yy

Uniformly loaded homogenious beam formula: <br> 
<center> $\delta_{max} = {5wL^4\over{384EI}}$</center><br> 
where: <br> 
<ul>
    <li> w - total weight </li>
    <li> L - span (watch your units) </li>
    <li> E - youngs modulus </li>
    <li> I - could be $I_{xx}$ or $I_{yy}$ depending on orientation. </li>
</ul>
<hr> 

In [4]:
def Deflection(load, Length, E, I):
    """
    Deflection in parent units from
    @param load in kips per inch
    @param Length in inches of span
    @param E - Youngs modulus in psi
    @param I - Area moment of inertia
    @return deflection in inches, note have to divide by 1000 to get units right
    Checked 02-Jul-24
    """
    delta = 5 * load *np.power(Length,4)/(384*E*I) * 1000
    print(" load ", '{:.4f}'.format(load), " kips/in, Length: ", Length, " Youngs ", E, " Area Inertia: ",'{:.4f}'.format(I), 
          " result = ", '{:.4f}'.format(delta))
    return delta


In [5]:
def BeamDeflection(W,H, Length, load):
    """
    Calculage deflection given a uniformly distributed load per unit length and supported at both ends.
    @param W - width
    @param H - height
    @param Length - span
    @param load - kips (kilo-lbs/in)
    Returns maximum deflection at Length/2
    TESTED: 01-Jul-24
    """
    YM_Wood = 9.0 # Gigapascal 9-14.5 for engineerd lumber
    # convert to PSI
    GPa2PSI = 145038
    E = YM_Wood * GPa2PSI
    I_xx,I_yy = AreaMomentBeam(W,H) # use I_yy since calculating the vertical deflection
    print("Engineered Lumber")
    delta = Deflection(load, Length, E, I_xx)
    return delta

In [6]:
def IBeamDeflection(H1, H2, a, b , Length, load):
    """
    This is for an I-beam, evenly distributed load and supported at ends. 
    @param H1 - outer face to face height
    @param H2 - web height
    @param a  - web thickness
    @param b  - base width
    @param Length - span
    @param load - load in kips (kilo-lbs/in)
    Returns Area moment of inertia section properties. in units^4 
    TESTED: 02-Jul-24
    """
    YM_Steel = 200     # GPa
    # convert to PSI
    GPa2PSI = 145038
    E = YM_Steel * GPa2PSI
    I_xx,I_yy = AreaMomentIBeam(H1, H2, a,b)
    print("steel I-beam, Ixx: ", '{:.4f}'.format(I_xx), " Iyy: ", '{:.4f}'.format(I_yy))
    delta = Deflection(load, Length, E, I_xx)
    return delta

In [7]:
# Problem
# Second floor area is ~1200 ft^2 half of which is supported by the outer wall. 
# The outer wall is ~35 4" feet
# The span we want is 12'
# household floor loading is 40lbs/ft^2 + weight of materials. Call it 60lbs/ft^2
Loading = 60        # pounds per square foot
Length  = 35.33     # feet
Width   = 13 + 8/12 # feet on the side of the house we are interested in. 
Area    = Length * Width 
Force   = Area * Loading # force on West side
print("Total weight: ", Force)
Force   = Force/2.0      # half of which is borne by center beam
kipsPERft = Force/Length/1000
kipsPERin = kipsPERft / 12
print("Force per beam: ", Force, " W: ", kipsPERft, " (kips/ft) ", kipsPERin," (kips/in)") 
#
# Assume this is linearly loaded. 
Span    = 12        # span in feet for the window. 
print("Span: ", Span, " feet")
Span = Span*12      # convert to inches
#
# Beam parameters
B_W     = 3.5       # beam width, constrained, it has to fit in the wall. 
B_H     = 12.0      # beam height
print("Beam Width: ", B_W, " inches, Height: ", B_H, "inches")
delta   = BeamDeflection(B_W, B_H, Span, kipsPERin)
# Tables for steel parameters
idx = 1
H1 = [6.0, 8.0, 12.0, 14.0, 24.0]
a  = [0.232, 0.271, 0.2, 0.25, 0.395]
b  = [3.332, 4.001, 3.97, 5.5, 7.0]
#
# steel i-beam 12" high
# a = 0.2
# b = 3.97
# d ends up as 3" over 12' span
#
# 14 inch high
# a = 0.250, b = 5.50
#
# 24" 
# a = 0.395
# b = 7.0
#
# these dimensions are pretty fixed

H2 = H1[idx] - 2*a[idx] # nominally
delta   = IBeamDeflection(H1[idx], H2, a[idx], b[idx] , Span, kipsPERin)
#print("Steel I-beam of height: ",H1," inches, Deflection is: ", delta, " inches, for a span of: ", Span, " inches")

Total weight:  28970.6
Force per beam:  14485.3  W:  0.41  (kips/ft)  0.034166666666666665  (kips/in)
Span:  12  feet
Beam Width:  3.5  inches, Height:  12.0 inches
Engineered Lumber
 load  0.0342  kips/in, Length:  144  Youngs  1305342.0  Area Inertia:  504.0000  result =  0.2908
steel I-beam, Ixx:  41.7672  Iyy:  2.9052
 load  0.0342  kips/in, Length:  144  Youngs  29007600  Area Inertia:  41.7672  result =  0.1579


In [8]:
x = Deflection(kipsPERin, Span, 1305342, 42.875)
d = 5 * kipsPERin * np.power(Span,4)/384/1305342/42.875
print(3.41793/x)

 load  0.0342  kips/in, Length:  144  Youngs  1305342  Area Inertia:  42.8750  result =  3.4179
1.0000005754668444


In [9]:
# quick calculation on wood member 7.5 x 1.5 inches 
# https://amesweb.info/Materials/Youngs-Modulus-of-Wood.aspx
YM_wood = 1.6e6        # psi
B_W     = 1.75         # beam width, constrained, it has to fit in the wall. 
B_H     = 12          # beam height
Span    = 77           # inches
WL      = 40.0 * 6 * 28/28/1000  # pounds per unit length 40lbs/sq/ft  28 foot span.  Should be in pounds
print("WL = ", WL)
WL  = WL/(28*12)       # pounds per inch, off by factor of 4??
delta   = BeamDeflection(B_W, B_H, Span, WL)
print("Joist wood Deflection is: ", delta, " inches, for a span of: ", Span, " inches")

WL =  0.24
Engineered Lumber
 load  0.0007  kips/in, Length:  77  Youngs  1305342.0  Area Inertia:  252.0000  result =  0.0010
Joist wood Deflection is:  0.0009939139641430424  inches, for a span of:  77  inches


In [10]:
Span/360 # rule of thumb for maximum allowable deflection 40lbs/sq ft

0.21388888888888888

In [11]:
#Plot both on the same basic parameters
load = np.arange(20,50,1) # pounds point load. 